In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/anotacoes/Anotacoes YOLO/val/labels/581825 (6).txt
/kaggle/input/anotacoes/Anotacoes YOLO/val/labels/581651 (3).txt
/kaggle/input/anotacoes/Anotacoes YOLO/val/labels/581831 (10).txt
/kaggle/input/anotacoes/Anotacoes YOLO/val/labels/581834 (5).txt
/kaggle/input/anotacoes/Anotacoes YOLO/val/labels/581660 (3).txt
/kaggle/input/anotacoes/Anotacoes YOLO/val/labels/581659 (5).txt
/kaggle/input/anotacoes/Anotacoes YOLO/val/labels/581832.txt
/kaggle/input/anotacoes/Anotacoes YOLO/val/labels/581657.txt
/kaggle/input/anotacoes/Anotacoes YOLO/val/labels/581833 (2).txt
/kaggle/input/anotacoes/Anotacoes YOLO/val/labels/1163464.txt
/kaggle/input/anotacoes/Anotacoes YOLO/val/labels/581658 (4).txt
/kaggle/input/anotacoes/Anotacoes YOLO/val/labels/581827 (7).txt
/kaggle/input/anotacoes/Anotacoes YOLO/val/labels/581829 (7).txt
/kaggle/input/anotacoes/Anotacoes YOLO/val/labels/581827 (1).txt
/kaggle/input/anotacoes/Anotacoes YOLO/val/labels/581661 (3).txt
/kaggle/input/anotacoes/Anotacoes Y

In [2]:
!pip install ultralytics

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 24.8 MB/s eta 0:00:0000:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.2 MB/s eta 0:00:000:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 1.8 MB/s eta 0:00:000:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.8 MB/s eta 0:00:000:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 30.8 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 13.5 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 7.5 MB/s eta 0:00:000:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 5.4 MB/s eta 0:00:000:00:0100:01
  Attempting uninstall: nvidia-nvjitlink-cu12
    Found existing installation: nvidia-nvjitlink-cu12 12.8.93
    Uninstalling nvidia-nvjitlink-cu12-12.8.93:
      Successfully uninstalled nvidia-nvjitlink-cu12-12.8.93
  Attempting uninstall: nvidia-curand-cu12
    Found existin

In [3]:
from ultralytics import YOLO
from PIL import Image
import matplotlib.pyplot as plt
import numpy as np

Creating new Ultralytics Settings v0.0.6 file ✅ 
View Ultralytics Settings with 'yolo settings' or at '/root/.config/Ultralytics/settings.json'
Update Settings with 'yolo settings key=value', i.e. 'yolo settings runs_dir=path/to/dir'. For help see https://docs.ultralytics.com/quickstart/#ultralytics-settings.


In [4]:
model = YOLO('yolov8n.pt')

100%|██████████| 6.25M/6.25M [00:00<00:00, 119MB/s]


In [5]:
# Cria o YAML com os caminhos corretos
yaml_content = """
train: /kaggle/input/anotacoes/Anotacoes YOLO/train/images
val: /kaggle/input/anotacoes/Anotacoes YOLO/val/images
test: /kaggle/input/anotacoes/Anotacoes YOLO/test/images

nc: 3
names: ['watertank', 'pool', 'waste']
"""

# Salva o arquivo no diretório atual
with open("custom_data.yaml", "w") as f:
    f.write(yaml_content)


In [ ]:
results = model.train(data="custom_data.yaml", epochs=50, imgsz=256)

Ultralytics 8.3.134 🚀 Python-3.11.11 torch-2.5.1+cu124 CUDA:0 (Tesla P100-PCIE-16GB, 16269MiB)
engine/trainer: agnostic_nms=False, amp=True, augment=False, auto_augment=randaugment, batch=16, bgr=0.0, box=7.5, cache=False, cfg=None, classes=None, close_mosaic=10, cls=0.5, conf=None, copy_paste=0.0, copy_paste_mode=flip, cos_lr=False, cutmix=0.0, data=custom_data.yaml, degrees=0.0, deterministic=True, device=None, dfl=1.5, dnn=False, dropout=0.0, dynamic=False, embed=None, epochs=50, erasing=0.4, exist_ok=False, fliplr=0.5, flipud=0.0, format=torchscript, fraction=1.0, freeze=None, half=False, hsv_h=0.015, hsv_s=0.7, hsv_v=0.4, imgsz=256, int8=False, iou=0.7, keras=False, kobj=1.0, line_width=None, lr0=0.01, lrf=0.01, mask_ratio=4, max_det=300, mixup=0.0, mode=train, model=yolov8n.pt, momentum=0.937, mosaic=1.0, multi_scale=False, name=train, nbs=64, nms=False, opset=None, optimize=False, optimizer=auto, overlap_mask=True, patience=100, perspective=0.0, plots=True, pose=12.0, pretrained

100%|██████████| 755k/755k [00:00<00:00, 24.2MB/s]


Overriding model.yaml nc=80 with nc=3

                   from  n    params  module                                       arguments                     
  0                  -1  1       464  ultralytics.nn.modules.conv.Conv             [3, 16, 3, 2]                 
  1                  -1  1      4672  ultralytics.nn.modules.conv.Conv             [16, 32, 3, 2]                
  2                  -1  1      7360  ultralytics.nn.modules.block.C2f             [32, 32, 1, True]             
  3                  -1  1     18560  ultralytics.nn.modules.conv.Conv             [32, 64, 3, 2]                
  4                  -1  2     49664  ultralytics.nn.modules.block.C2f             [64, 64, 2, True]             
  5                  -1  1     73984  ultralytics.nn.modules.conv.Conv             [64, 128, 3, 2]               
  6                  -1  2    197632  ultralytics.nn.modules.block.C2f             [128, 128, 2, True]           
  7                  -1  1    295424  ultralytics

100%|██████████| 5.35M/5.35M [00:00<00:00, 103MB/s]


AMP: checks passed ✅
train: Fast image access ✅ (ping: 0.1±0.2 ms, read: 31.7±11.0 MB/s, size: 131.6 KB)


...


      Epoch    GPU_mem   box_loss   cls_loss   dfl_loss  Instances       Size


      26/50      0.57G      1.209      0.857      1.032         22        256: 100%|██████████| 59/59 [00:05<00:00, 11.74it/s]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 8/8 [00:00<00:00,  8.61it/s]

                   all        232        514      0.714      0.621      0.715      0.416



      Epoch    GPU_mem   box_loss   cls_loss   dfl_loss  Instances       Size


      27/50      0.57G      1.178     0.8837       1.03         47        256:  47%|████▋     | 28/59 [00:02<00:02, 11.53it/s]

In [ ]:
best_model = YOLO('/kaggle/working/runs/detect/train/weights/best.pt')

In [ ]:
from ultralytics import YOLO
from PIL import Image  # Importar PIL.Image para fromarray
import matplotlib.pyplot as plt

In [ ]:
#image_paths = '/kaggle/input/inferencia/1163357.png'
image_paths = '/kaggle/input/inferencia/1163373.png' 
#image_paths = '/kaggle/input/inferencia/1163376.png'
#image_paths = "//kaggle/input/inferencia/1163631.png" #não detectada
#image_paths = '/kaggle/input/inferencia/1163683.png'


results = best_model.predict(image_paths, conf=0.5, imgsz=256)

annotated_img = results[0].plot()  # Caixas delimitadoras com array NumPy)
annotated_img_pil = Image.fromarray(annotated_img)  #PIL.Image.fromarray para transformar em img

plt.figure(figsize=(8, 8))
plt.imshow(annotated_img_pil)
plt.show()

print(f'Imagem: {image_paths}\n')

In [ ]:
for i, result in enumerate(results[0].boxes):  # Itera sobre as caixas detectadas

    box_coords = result.xyxy[0].cpu().numpy()  # Converte para NumPy
    x_min, y_min, x_max, y_max = box_coords
    
    # Extrai a confiança como um valor escalar
    confidence = result.conf.cpu().numpy().item()  # Usa .item() para obter o valor escalar
    
    # Extrai o ID da classe como um valor escalar
    class_id = int(result.cls.cpu().numpy().item())  # Usa .item() para obter o valor escalar
    class_name = results[0].names[class_id]  # Nome da classe

    print(f'Objeto {i+1}:')
    print(f'  Classe: {class_name}')
    print(f'  Coordenadas (x_min, y_min, x_max, y_max): ({x_min:.2f}, {y_min:.2f}, {x_max:.2f}, {y_max:.2f})\n')

In [ ]:
!pip install opencv-python

In [ ]:
import cv2

In [ ]:
image_paths = '/kaggle/input/inferencia/1163683.png'


In [ ]:
# Realiza a predição
results = best_model.predict(image_paths, conf=0.5, imgsz=256)

# Obtém a imagem anotada com as caixas delimitadoras
annotated_img = results[0].plot()  # Caixas delimitadoras com array NumPy
annotated_img_rgb = cv2.cvtColor(annotated_img, cv2.COLOR_BGR2RGB)
annotated_img_pil = Image.fromarray(annotated_img_rgb)

# Exibe a imagem com as caixas delimitadoras
plt.figure(figsize=(8, 8))
plt.imshow(annotated_img_pil)
plt.axis('off') 
plt.show()

# Lista para armazenar os dados que serão salvos no CSV
data = []

# Extrai e imprime as coordenadas das caixas delimitadoras
for i, result in enumerate(results[0].boxes):  # Itera sobre as caixas detectadas
    box_coords = result.xyxy[0].cpu().numpy()  # Converte para NumPy
    x_min, y_min, x_max, y_max = box_coords
    
    # Extrai o ID da classe como um valor escalar
    class_id = int(result.cls.cpu().numpy().item())  # Usa .item() para obter o valor escalar
    class_name = results[0].names[class_id]  # Nome da classe

    # Imprime os resultados
    print(f'Objeto {i+1}:')
    print(f'  Classe: {class_name}')
    print(f'  Coordenadas (x_min, y_min, x_max, y_max): ({x_min:.2f}, {y_min:.2f}, {x_max:.2f}, {y_max:.2f})\n')

    # Adiciona os dados à lista
    data.append({
        'image_path': image_paths,
        'object_id': i + 1,
        'class': class_name,
        'x_min': x_min,
        'y_min': y_min,
        'x_max': x_max,
        'y_max': y_max
    })

# Cria um DataFrame com os dados
df = pd.DataFrame(data)

# Salva o DataFrame em um arquivo CSV no diretório /kaggle/working/
output_csv = '/kaggle/working/coordenadas.csv'
df.to_csv(output_csv, index=False)
